In [1]:
import matplotlib.pyplot as plt
import networkx as nx
from graphviz import Digraph, Graph
params, section = ['name', 'sequence', 'optional', 'quality'], 4
fn = '6/s_6.first1000.fastq'#'6/s_6.first10000.fastq'#

In [6]:
class Node():
    def __init__(self, value=None):
        self.value = value
        
    def get_value(self):
        return self.value

class DeBruijnGraph(Graph):
    def __init__(self):
        self.graph = {}
        self.nodes = {}
        self.average_kmers = {}
        self.edgeWeights = {}
    
    def add_vertex(self, node_str):
        # this adds node as a vertex to graph
        if node_str not in self.nodes:
            node = Node(node_str)
            self.nodes[node_str] = node
            self.graph[node] = []
            return node
        else: return self.nodes[node_str]

    def add_edge(self, node1, node2):
        # adds an edge between node1 and node2
        prefix_node = self.add_vertex(node1)
        suffix_node = self.add_vertex(node2)
        self.graph[prefix_node] += [suffix_node]
        index = prefix_node.get_value() + suffix_node.get_value()[-1]
        self.edgeWeights[index] = self.edgeWeights.get(index, 0) + 1
        # add length to each time that this weight is updated.
        
    def first_node(self, node):
        suffix_node = self.add_vertex(node)
        count = [1 for n, value in self.graph.items() if suffix_node in value]
        return sum(count) == 0
    
    def get_graph(self):
        return {k.get_value():[v.get_value() for v in value] for k, value in self.graph.items()}
    
    def make_graph(self, genome, k=3):
        for kmer in kmer_generator(genome, k):
            nodes = [i for i in kmer_generator(kmer, k-1)]
            self.add_edge(nodes[0], nodes[1])
            if nodes[0] in self.average_kmers:
                self.average_kmers[nodes[0]] += 1
            else:
                self.average_kmers[nodes[0]] = 1

            if nodes[1] in self.average_kmers:
                self.average_kmers[nodes[1]] += 1
            else:
                self.average_kmers[nodes[1]] = 1

class CondensedDeBruijnGraph(DeBruijnGraph):
    def __init__(self):
        self.graph = {} #from non-condensed
        self.nodes = {} #from non-condensed
        self.edgeWeights = {} #from non-condensed
        self.average_kmers = {}
        self.condensedWeights = {}
        self.condensedNodes = {}
        self.condensedGraph = {}
    
    def get_edgeW(self, edge):
        return self.condensedWeights[edge]
    
    def get_nodes(self):
        return list(self.condensedNodes.keys())
    
    def get_edges(self):
        for node, value in self.condensedGraph.items():
            for suffix_node in value:
                yield (node.get_value(), suffix_node.get_value())
    
    def get_condensed_nodes(self):
        for node in self.nodes: 
            prefix_node = self.add_vertex(node)
            if len(self.graph[prefix_node])==0: # if this is the last element in the strand
                self.condensedNodes[node] = prefix_node
            elif len(self.graph[prefix_node])>1: # if this element has a branch, \
                #so number of edges leaving is greater than 1 (or entered from more than 1)
                self.condensedNodes[node] = prefix_node
            elif self.first_node(node): #this is the first element in the strand
                self.condensedNodes[node] = prefix_node
        return list(self.condensedNodes.keys())

    def recurEdge(self, suffix_node, edge_label, weight):
        edge_label += suffix_node.get_value()[-1]
        if suffix_node.get_value() in self.condensedNodes:
            self.condensedNodes[suffix_node.get_value()] = suffix_node
            return suffix_node, edge_label, weight
        else:
            suffix_node_array = self.graph[suffix_node]
            index = suffix_node.get_value() + suffix_node_array[0].get_value()[-1]
            weight += self.edgeWeights[index]
            return self.recurEdge(suffix_node_array[0], edge_label, weight)
    
    def make_condensed(self):
        cond_nodes = self.get_condensed_nodes()
        for node in cond_nodes:
            prefix_node = self.add_vertex(node)
            self.condensedNodes[node] = prefix_node
            suffix_node_array = self.graph[prefix_node]     
            for suffix_node in suffix_node_array:
                index = node + suffix_node.get_value()[-1]
                weight = self.edgeWeights[index]
                cond_node, edge_label, weight = self.recurEdge(suffix_node, node, weight)
                self.condensedGraph.setdefault(prefix_node, []).append(cond_node)
                index = node + cond_node.get_value()[-1]
                if not index in self.condensedWeights:
                    self.condensedWeights[index] = edge_label, weight

In [7]:
def get_reads(fn):
    with open(fn, 'r') as file:
        lines = []
        for line in file:
            lines.append(line.rstrip())
            if len(lines) == section:
                record = {k: v for k, v in zip(params, lines)}
                lines = []
                yield record['sequence']

# write k-mer of graph
def kmer_generator(genome, k=3):
    i = 0
    while i + k <= len(genome):
        kmer = genome[i:i+k]
        yield kmer
        i += 1

In [14]:
# Condensed De Brujin Analysis
k, j = 55, 0
genome = ['AAACTTATGGGACCCA', 'AAACGGGATTATCCCA']
graph = CondensedDeBruijnGraph()
for genome in get_reads(fn): #genome: #
    graph.make_graph(genome, k)
    j+=1
graph.make_condensed()

Done


In [15]:
all_edges = [(i,v) for i, v in graph.get_edges()]
all_edges = list(set(all_edges))
all_nodes = graph.get_nodes()
dot = Digraph(format='png')
dict_keys = {all_nodes[i]:str(i) for i in range(len(all_nodes)) }
[dot.node(dict_keys[i], i) for i in all_nodes] #for i, v in graph.get_edges() if v is None]
[dot.edge(dict_keys[i], dict_keys[v]) for i, v in all_edges]
# [dot.edge(dict_keys[i], dict_keys[v], 
#           label=graph.get_edgeW(i+v[-1])[0]+ ' (' +str(graph.get_edgeW(i+v[-1])[1])+')'+str(avg_reads[i+v[-1]]))\
#                                                          for i, v in all_edges]
dot.render('graph4', view=True)

'graph4.png'

In [11]:
DG = nx.MultiDiGraph()
DG.add_nodes_from(all_nodes)
DG.add_edges_from(all_edges)
# plt.figure(figsize=(10,10))
# pos = nx.spring_layout(DG,scale=2)
# nx.draw(DG, pos, with_labels=True)
# nx.draw_networkx_edge_labels(DG, pos=pos, edge_labels={i: str(graph.get_edgeW(i[0]+i[1][-1]))\
#                                 + ' (' + '{0:.2f}'.format(avg_reads[i[0]+i[1][-1]]) + ')' for i in all_edges})
# plt.show()

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [12]:
from networkx.drawing.nx_agraph import write_dot
write_dot(DG,'k55_new.dot')
# !neato -T png multi.dot > multi.png

It is not clear what the average kmer calculation is for the edge and the tip removal exercise depends on accurately calculating this.